In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import torch.nn
import torch

/usr/local/var/pyenv/versions/py3/envs/ML/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [40]:
class ConvRF(object):
    def __init__(self, kernel_size=5, stride=2):
        self.kernel_size = kernel_size
        self.stride = stride
        self.kernel_forests = None
        self.num_outputs = 10

    def _convolve(self, images, labels=None, flatten=False):

        batch_size, in_dim, _, num_channels = images.shape

        out_dim = int((in_dim - self.kernel_size) / self.stride) + 1  # calculate output dimensions

        # create matrix to hold the chopped images
        out_images = np.zeros((batch_size, out_dim, out_dim,
                               self.kernel_size, self.kernel_size, num_channels))
        out_labels = None

        curr_y = out_y = 0
        # move kernel vertically across the image
        while curr_y + self.kernel_size <= in_dim:
            curr_x = out_x = 0
            # move kernel horizontally across the image
            while curr_x + self.kernel_size <= in_dim:
                # chop images
                out_images[:, out_x, out_y] = images[:, curr_x:curr_x +
                                                     self.kernel_size, curr_y:curr_y+self.kernel_size, :]
                curr_x += self.stride
                out_x += 1
            curr_y += self.stride
            out_y += 1

        if flatten:
            out_images = out_images.reshape(batch_size, out_dim, out_dim, -1)

        if labels is not None:
            out_labels = np.zeros((batch_size, out_dim, out_dim))
            out_labels[:, ] = labels.reshape(-1, 1, 1)

        return out_images, out_labels, out_dim

    def convolve_fit(self, images, labels):
        sub_images, sub_labels, out_dim = self._convolve(images, labels=labels, flatten=True)

        self.kernel_forests = [[0]*out_dim for _ in range(out_dim)]
        convolved_image = np.zeros((images.shape[0], out_dim, out_dim, self.num_outputs))
        for i in range(out_dim):
            for j in range(out_dim):
                self.kernel_forests[i][j] = RandomForestClassifier()
                self.kernel_forests[i][j].fit(sub_images[:, i, j], sub_labels[:, i, j])
#                 convolved_image[:, i, j] = self.kernel_forests[i][j].predict_proba(sub_images[:, i, j])
                convolved_image[:, i, j] = self.kernel_forests[i][j].apply(sub_images[:, i, j])

        return convolved_image

    def convolve_predict(self, images):
        if not self.kernel_forests:
            raise Exception("Should fit training data before predicting")

        sub_images, _, out_dim = self._convolve(images, flatten=True)

        kernel_predictions = np.zeros((images.shape[0], out_dim, out_dim, self.num_outputs))
        for i in range(out_dim):
            for j in range(out_dim):
#                 kernel_predictions[:, i, j] = self.kernel_forests[i][j].predict_proba(sub_images[:, i, j])
                kernel_predictions[:, i, j] = self.kernel_forests[i][j].apply(sub_images[:, i, j])

        return kernel_predictions

In [35]:
# prepare MNIST data
import torchvision.datasets as datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_train_images = mnist_trainset.train_data.numpy()[..., np.newaxis]
mnist_train_labels = mnist_trainset.train_labels.numpy()

# X_train, X_test, y_train, y_test = train_test_split(mnist_train_images, mnist_train_labels,
#                                                     stratify=mnist_train_labels,
#                                                     test_size=0.25)
# mnist_train_images = X_train
# mnist_train_labels = y_train
# mnist_test_images = X_test
# mnist_test_labels = y_test

mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)
mnist_test_images = mnist_testset.test_data.numpy()[..., np.newaxis]
mnist_test_labels = mnist_testset.test_labels.numpy()

In [54]:
def run_experiment(mnist_train_images, mnist_train_labels, mnist_test_images, mnist_test_labels):
    print("Num. of Convolution Layers: 1")
    # conv layer 1
    conv1 = ConvRF(kernel_size=3, stride=2)
    conv1_map = conv1.convolve_fit(mnist_train_images, mnist_train_labels)
    
    # full RF (conv 1)
    conv1_full_RF = RandomForestClassifier(n_estimators=100)
    conv1_full_RF.fit(conv1_map.reshape(len(mnist_train_images), -1), mnist_train_labels)
    
    # test (conv 1)
    conv1_map_test = conv1.convolve_predict(mnist_test_images)
    mnist_test_preds = conv1_full_RF.predict(conv1_map_test.reshape(len(mnist_test_images), -1))

    print("Test Accuracy: " + str(accuracy_score(mnist_test_labels, mnist_test_preds)))
    print("Validation Confusion Matrix: \n" + str(confusion_matrix(mnist_test_labels, mnist_test_preds)))
    
#     # max pool layer
#     m = torch.nn.MaxPool2d(3, stride=1, padding=1)
#     conv1_map = m(torch.from_numpy(conv1_map)).numpy()
#     conv1_map_test = m(torch.from_numpy(conv1_map_test)).numpy()

    # relu
    m = torch.nn.ReLU()    
    conv1_map = m(torch.from_numpy(conv1_map)).numpy()
    conv1_map_test = m(torch.from_numpy(conv1_map_test)).numpy()

    print("Num. of Convolution Layers: 2")
    # conv layer 2
    conv2 = ConvRF(kernel_size=3, stride=2)
    conv2_map = conv2.convolve_fit(conv1_map, mnist_train_labels)
    
    # full RF (conv 2)
    conv2_full_RF = RandomForestClassifier(n_estimators=100)
    conv2_full_RF.fit(conv2_map.reshape(len(mnist_train_images), -1), mnist_train_labels)
    
    # test (conv 2)
    conv2_map_test = conv2.convolve_predict(conv1_map_test)
    mnist_test_preds = conv2_full_RF.predict(conv2_map_test.reshape(len(mnist_test_images), -1))

    print("Test Accuracy: " + str(accuracy_score(mnist_test_labels, mnist_test_preds)))
    print("Validation Confusion Matrix: \n" + str(confusion_matrix(mnist_test_labels, mnist_test_preds)))

    # relu
    m = torch.nn.ReLU()    
    conv2_map = m(torch.from_numpy(conv2_map)).numpy()
    conv2_map_test = m(torch.from_numpy(conv2_map_test)).numpy()
    
    print("Num. of Convolution Layers: 3")
    # conv layer 3
    conv3 = ConvRF(kernel_size=3, stride=1)
    conv3_map = conv3.convolve_fit(conv2_map, mnist_train_labels)
    
    # full RF (conv 3)
    conv3_full_RF = RandomForestClassifier(n_estimators=100)
    conv3_full_RF.fit(conv3_map.reshape(len(mnist_train_images), -1), mnist_train_labels)
    
    # test (conv 3)
    conv3_map_test = conv3.convolve_predict(conv2_map_test)
    mnist_test_preds = conv3_full_RF.predict(conv3_map_test.reshape(len(mnist_test_images), -1))

    print("Test Accuracy: " + str(accuracy_score(mnist_test_labels, mnist_test_preds)))
    print("Validation Confusion Matrix: \n" + str(confusion_matrix(mnist_test_labels, mnist_test_preds)))

In [37]:
# when using rf.predict_proba() - 3x3 kernel, with 2 stride, 3x3 kernel stride 1 conv 3
run_experiment(mnist_train_images, mnist_train_labels, mnist_test_images, mnist_test_labels)

Num. of Convolution Layers: 1
Test Accuracy: 0.924
Validation Confusion Matrix: 
[[ 956    0    4    2    1    2    9    1    3    2]
 [   0 1114    7    1    0    1    5    2    5    0]
 [  13    2  952   16    2    0    6   16   24    1]
 [   4    1   21  918    0   32    3   10   16    5]
 [   1    7   10    0  912    1    6    1    5   39]
 [   7    2    1   42    1  811   11    4    9    4]
 [  14    3    4    2    5   16  912    0    1    1]
 [   3   12   38    3    8    3    0  910    5   46]
 [   9    2   26   34    4   28    7    9  843   12]
 [  13    6    7    7   31    3    0   22    8  912]]
Num. of Convolution Layers: 2
Test Accuracy: 0.8475
Validation Confusion Matrix: 
[[ 849    0    6   24    2   24   54   14    4    3]
 [   0 1091   14    3    2    3    5    2   13    2]
 [  23   30  837   45   13    9   14   30   25    6]
 [   9    1   15  830    0  116    3   16   12    8]
 [   3   14   16    2  831    7   18    8    8   75]
 [  11    2    2   99    5  730   12    7

In [11]:
# when using rf.apply() - 3x3 kernel, with 2 stride, 3x3 kernel stride 1 conv 3
run_experiment(mnist_train_images, mnist_train_labels, mnist_test_images, mnist_test_labels)

Num. of Convolution Layers: 1
Test Accuracy: 0.9685
Validation Confusion Matrix: 
[[ 970    1    0    0    1    4    1    1    1    1]
 [   0 1120    3    4    0    1    1    1    4    1]
 [   6    0 1000    3    5    0    1    9    8    0]
 [   0    1   12  969    0    6    0    9    8    5]
 [   1    0    0    0  956    0    5    0    5   15]
 [   4    1    0   13    3  852    8    1    5    5]
 [   4    2    1    0    2    6  938    0    4    1]
 [   2    4   11    1    2    0    0  992    5   11]
 [   5    0    4    7    6    5    6    3  928   10]
 [   5    4    3    7   14    2    1    4    9  960]]
Num. of Convolution Layers: 2
Test Accuracy: 0.9589
Validation Confusion Matrix: 
[[ 966    1    0    0    2    2    2    2    3    2]
 [   0 1116    2    5    1    1    4    1    5    0]
 [   4    0 1001    2    3    2    1    9   10    0]
 [   0    1   15  949    0   18    0    9   11    7]
 [   1    0    3    0  947    0    5    1    7   18]
 [   4    0    1   27    3  828   11    

In [25]:
# when using rf.apply() - 5x5 kernel, with 2 stride
run_experiment(mnist_train_images, mnist_train_labels, mnist_test_images, mnist_test_labels)

Num. of Convolution Layers: 1
Test Accuracy: 0.9669
Validation Confusion Matrix: 
[[ 971    0    0    0    0    4    1    1    2    1]
 [   0 1120    5    2    0    1    1    1    4    1]
 [   5    0 1004    4    1    1    1    8    8    0]
 [   0    0   12  969    0    8    0    9    7    5]
 [   1    0    2    0  951    0    5    0    4   19]
 [   3    1    1   22    4  841    9    1    5    5]
 [   5    2    1    0    3    7  935    0    5    0]
 [   1    3   13    2    1    0    0  996    4    8]
 [   3    1    5    8    6    6    7    4  926    8]
 [   6    5    1   11   16    3    1    5    5  956]]
Num. of Convolution Layers: 2
Test Accuracy: 0.9485
Validation Confusion Matrix: 
[[ 966    1    1    0    0    3    2    1    3    3]
 [   0 1114    2    6    0    2    2    2    7    0]
 [   5    2  980    8    4    4    5   14    9    1]
 [   1    0   20  928    0   27    1   17   13    3]
 [   3    0    2    1  926    2    5    0   10   33]
 [   6    4    4   30    3  812   11    

ValueError: cannot reshape array of size 0 into shape (60000,0,0,newaxis)

In [6]:
# when using rf.apply() - 3x3 kernel with 2 stride conv 1, 1x1 kernel, with 1 stride for conv 2 and 3
run_experiment(mnist_train_images, mnist_train_labels, mnist_test_images, mnist_test_labels)

Num. of Convolution Layers: 1
Test Accuracy: 0.9688
Validation Confusion Matrix: 
[[ 970    0    2    0    0    2    2    1    2    1]
 [   0 1121    3    2    0    1    1    2    5    0]
 [   4    0 1001    2    5    0    2    7   10    1]
 [   0    1   10  971    0    9    0    9    6    4]
 [   1    0    1    0  955    0    5    0    6   14]
 [   2    1    0   11    5  854    8    2    5    4]
 [   4    3    0    0    3    5  937    0    6    0]
 [   2    1   12    1    2    0    0  994    3   13]
 [   3    0    5    6    6   10    5    4  926    9]
 [   4    5    3    9   17    3    1    5    3  959]]
Num. of Convolution Layers: 2
Test Accuracy: 0.9681
Validation Confusion Matrix: 
[[ 972    0    1    0    0    2    1    1    2    1]
 [   0 1121    2    6    0    2    1    1    1    1]
 [   5    1 1000    5    4    0    1    8    8    0]
 [   0    0    9  970    0    8    0   10    8    5]
 [   1    0    0    0  954    0    5    1    5   16]
 [   3    2    0   18    5  843    6    

In [24]:
# test PyTorch max pooling
import torch.nn
m = torch.nn.MaxPool2d(3, stride=1)
input = torch.randn(20, 10, 16, 50)
output = m(np.random.randn((10,10,10,10)))
output.shape

torch.Size([20, 10, 14, 48])

In [52]:
# when using rf.apply() - max pool after conv 1
run_experiment(mnist_train_images, mnist_train_labels, mnist_test_images, mnist_test_labels)

Num. of Convolution Layers: 1
Test Accuracy: 0.9689
Validation Confusion Matrix: 
[[ 971    0    1    0    0    2    1    1    3    1]
 [   0 1120    3    3    0    2    1    1    4    1]
 [   5    0 1002    4    5    0    1    8    7    0]
 [   0    0   10  971    0    6    0    9   11    3]
 [   1    0    1    0  954    0    5    0    4   17]
 [   3    0    1   15    3  848    8    1    8    5]
 [   5    2    1    0    2    5  940    0    3    0]
 [   2    2   12    0    1    0    0  995    4   12]
 [   2    0    6    9    8    5    5    3  925   11]
 [   6    4    3   11   11    1    1    3    6  963]]
Num. of Convolution Layers: 2
Test Accuracy: 0.9554
Validation Confusion Matrix: 
[[ 963    0    0    1    3    4    3    1    5    0]
 [   0 1116    5    3    1    1    3    1    5    0]
 [   3    1 1001    6    2    1    2    8    7    1]
 [   1    0   13  959    0   12    0   15    7    3]
 [   1    0    4    2  931    0    6    2    7   29]
 [   6    3    1   32    8  815    8    

In [55]:
# when using rf.apply() - relu after conv 1 and conv 2
run_experiment(mnist_train_images, mnist_train_labels, mnist_test_images, mnist_test_labels)

Num. of Convolution Layers: 1
Test Accuracy: 0.9701
Validation Confusion Matrix: 
[[ 972    0    0    0    0    2    2    1    2    1]
 [   0 1121    3    5    0    2    1    1    2    0]
 [   6    0 1001    2    5    0    1    9    8    0]
 [   0    0   10  973    0    7    0    8    9    3]
 [   1    0    1    0  957    0    4    0    4   15]
 [   3    0    1   17    3  850    8    1    4    5]
 [   5    3    1    0    4    4  935    0    6    0]
 [   2    1   10    0    1    0    0  999    5   10]
 [   5    0    6    4    6    6    3    3  929   12]
 [   4    4    1    8   15    2    1    4    6  964]]
Num. of Convolution Layers: 2
Test Accuracy: 0.9594
Validation Confusion Matrix: 
[[ 969    0    1    0    1    2    2    1    2    2]
 [   0 1115    3    5    0    1    3    2    6    0]
 [   3    0  997    5    3    1    2    9   11    1]
 [   1    2   14  958    0   10    0   13    9    3]
 [   2    1    4    0  937    0    5    0    9   24]
 [   4    0    2   25    4  833    8    